In [ ]:
from openai import OpenAI, AzureOpenAI
import json
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
AZURE_API_KEY = os.getenv("AZURE_API_KEY")
AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")
AZURE_OPENAI_VERSION=os.getenv("AZURE_OPENAI_VERSION")
client=OpenAI(api_key=OPENAI_API_KEY)
aclient=AzureOpenAI(api_key=AZURE_API_KEY, azure_endpoint=AZURE_ENDPOINT, api_version=AZURE_OPENAI_VERSION)

In [ ]:
import csv
assistants = []
# read assistants in from assistants.csv
with open('assistants.csv', 'r') as f:
  reader = csv.DictReader(f)
  for row in reader:
      print(row['assistant_id'], row['name'])
      assistants.append(row)

In [ ]:
for row in assistants:
    print(row['assistant_id'], row['name'])

In [ ]:
oassistant=client.beta.assistants.retrieve(assistants[0]['assistant_id'])

In [ ]:
print(oassistant.tool_resources.file_search.vector_store_ids)
vector_store_files=client.beta.vector_stores.files.list(oassistant.tool_resources.file_search.vector_store_ids[0])
print(vector_store_files)

In [ ]:
#manually set assistants
# assistants=[{'assistant_id':"asst_OVshFbXYzcx2DOpSVlUb8T0H", 'name':"Article analysis bot"}]

In [ ]:
import pprint
for assistant in assistants:

    oassistant=client.beta.assistants.retrieve(assistant['assistant_id'])
    print(oassistant.id)
    print(oassistant.name)
    print(oassistant.model)
    print(oassistant.instructions[:100])
    print(oassistant.tools)
    print(oassistant.tool_resources)
    vector_store_files=client.beta.vector_stores.files.list(oassistant.tool_resources.file_search.vector_store_ids[0])
    print("-----------------")

In [ ]:
#write to azure
for assistant in assistants:
    oassistant=client.beta.assistants.retrieve(assistant['assistant_id'])
    aassistant=aclient.beta.assistants.create(
        instructions=oassistant.instructions,
        name=oassistant.name,
        # tools=oassistant.tools,
        model="gpt-4o"
    )
    print(aassistant.id)
    print(aassistant)
    input("Press Enter to continue...")

In [ ]:
new_assistant=[]
azure_list=aclient.beta.assistants.list()
for assistant in azure_list:
    print(assistant.id)
    print(assistant.name)
    print(assistant.model)
    print(assistant.instructions[:100])
    print(assistant.tools)
    print("-----------------")
    new_assistant.append({"assistant_id":assistant.id, "name":assistant.name})

In [ ]:
for assistant in new_assistant:
    print(assistant['assistant_id'], assistant['name'])

In [ ]:
# iterate through new_assistant and write to csv if name contains "Study
with open('azure_assistants.csv', 'w', newline='') as f:
    fieldnames = ['assistant_id', 'name']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for assistant in new_assistant:
        if "Study" in assistant['name']:
            writer.writerow(assistant)
            print(assistant['assistant_id'], assistant['name'])
            print

In [ ]:
print(new_assistant[0])